# Code and plots for Wind Lectures


The selected location is -38.51226,147.75921, Near the centre of the OEI-01-2022 Part 1 (Gippsland) Offshore Renewable Energy Infrastructure Region [link](https://geoscience-au.maps.arcgis.com/apps/Styler/index.html?appid=5d4b8f1458ee4a12a2070cdd943c0f88). See CSV for location. The data is obtained using the "Single Point" option from the [NASA Power DAV](https://power.larc.nasa.gov/data-access-viewer/).

In [ ]:
import enn554.wind as wind
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.img_tiles import OSM
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from shapely.geometry import box, Point
import cartopy.mpl.ticker as cticker
from matplotlib.ticker import FormatStrFormatter
from enn554.paths import data_dir, repo_root, package_root # this is the path to the data directory\
dd = data_dir()

In [ ]:
gdf = gpd.read_file(dd/'OffshoreRenewable_Energy_Infrastructure_Regions/offshore_renewable_energy_infrastructure_regions.shp')
gdf = gdf.to_crs(epsg=4326)
gdf = gdf[gdf["Status"] == "Declared"] # exclude superceded regions
df = pd.read_csv(dd/'gippsland_offshore.csv',skiprows=14)
merra_data = wind.merra_wind_speed_data()
merra_data.import_data(dd/'gippsland_offshore.csv')
# lat0,lon0 = merra_data.latitude,merra_data.longitude

In [ ]:
dist,fig,ax = wind.speed_fit(merra_data.data['WS50M'],type='Weibull',plot=True,hist_kwargs={'bins':50})
dist_r = wind.speed_fit(merra_data.data['WS50M'],type='Rayleigh',plot=False)

x = np.linspace(ax.get_xlim()[0],ax.get_xlim()[1],1000)
ax.plot(x,dist_r.pdf(x),'r--',label='Rayleigh')
ax.legend()

# Directional distribution fit to Gippsland data

In [ ]:
N = 36
speed,direction = merra_data.data['WS50M'].values,merra_data.data['WD50M'].values
azimuth_edges = np.linspace(0,360,N)
wdist = wind.speed_and_direction_dist(type='Weibull',n_az_bins=N)
wdist.fit(direction,speed)

## Wind rose with fit
In the below, the angle corresponds to the direction that the wind is *coming from*, the radius corresponds to percentage of observations that are coming from that direction and the stacked colors represent the wind speed proportion in that direction. 

In [ ]:
ax = wind.wind_rose(direction,speed,num_bins=N,s_units='m/s')
ax.plot(np.pi/2.0-np.pi/180*wdist.azimuth_bin_centers,wdist.probabilities*100,'r.',label='Estimated probabilities')

## Histograms with fit
First histogram is the total data and the second one is for a specific direction. 

In [ ]:
fig,ax = plt.subplots()
ax.hist(speed,density=True,bins=50,label=f'Data (N={len(speed)})')
x = np.linspace(ax.get_xlim()[0],ax.get_xlim()[1],1000)
ax.plot(x,wdist.pdf(x),'r--',label='Estimated distribution')
ax.set_xlabel('Wind speed [m/s]')
ax.set_ylabel('Probability density')
ax.legend()

In [ ]:
direction_of_interest = 250 # degrees from north, clockwise

idx = np.digitize(direction_of_interest,wdist.azimuth_bin_edges,right=True)
dir_bins = np.digitize(direction,wdist.azimuth_bin_edges,right=True)
dir_bins[dir_bins==len(wdist.azimuth_bin_edges)] = 0
if idx == len(wdist.azimuth_bin_edges):
    idx = 0

fig,ax = plt.subplots()
ax.hist(speed[dir_bins==idx],density=True,label=f'Data (N={np.sum(dir_bins==idx)})')
x = np.linspace(ax.get_xlim()[0],ax.get_xlim()[1],1000)
ax.plot(x,wdist.dists[idx].pdf(x),'r--',label='Estimated distribution')
ax.set_xlabel('Wind speed [m/s]')
ax.set_ylabel('Probability density')
ax.set_title(f'Wind speed {direction_of_interest} degrees CW from north')
ax.legend()

## Individual direction distributions

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
x = np.linspace(min(speed),max(speed),1000)
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection='3d')
for ii,d in enumerate(wdist.dists):
    # ax.plot(x,d.pdf(x),label=f'direction = {wdist.azimuth_bin_centers[ii]:.1f}')
    y = wdist.azimuth_bin_centers[ii]*np.ones_like(x)
    ax.plot3D(x,y,d.pdf(x))
    ax.set_xlabel('Wind speed [m/s]')
    ax.set_ylabel('Wind direction [deg CW from N]')
    ax.set_zlabel('Probability density')

# Power curve importing
See [NREL/turbine-models](https://github.com/NREL/turbine-models/tree/main) GitHub for models. 

### Listing all turbines of a particular type using NREL's turbine modles

In [ ]:
from turbine_models.parser import Turbines
turb = Turbines()
turb.turbines(group='offshore')

In [ ]:
power_curve = wind.turbine()
power_curve.import_nrel_power_curve('IEA_Reference_15MW_240')
power_curve.plot(nonzero_only=False)

# Power for selected location and turbine power curve

In [ ]:
units = "MW" # MW, kW, W
power,prob = wind.power_cdf(wdist,power_curve,n_bins=1000,power_units=units)
fig,ax = plt.subplots()
ax.step(power,prob,where='post')
ax.set_xlabel(f'Power [{units}]')
ax.set_ylabel('Power CDF')
ax.set_ylim((-0.01,1.01))
ax.grid(True)

# Aside: Testing the fitting of the directional Weibull

In [ ]:
import scipy.stats as stats

In [ ]:
N = 5
probabilities = stats.binom(N-1,0.5).pmf(np.arange(N))
wdist2 = wind.speed_and_direction_dist( type='Weibull',
                                        shapes=[2,2,3,4,4],
                                        scales=[10,5,30,30,30],
                                        n_az_bins=N,
                                        probabilities=probabilities)

fig,ax = plt.subplots(nrows=2)
dirs,speeds = wdist2.rvs(1000)
ax[0].hist(speeds,density=True,label='Simulated',bins=None)
x = np.linspace(ax[0].get_xlim()[0],ax[0].get_xlim()[1],1000)
ax[0].plot(x,wdist2.pdf(x),'r--',label='True distribution')
values,counts = np.unique(dirs,return_counts=True)
ax[1].bar(x=wdist2.azimuth_bin_centers,width=360/(N+1),height=counts/sum(counts),label='Simulated')
ax[1].bar(x=wdist2.azimuth_bin_centers,height=probabilities,width=360/(N+1),color="red",alpha=0.5,label='True distribution')

In [ ]:
wdist_fitted = wind.speed_and_direction_dist(type='Weibull',n_az_bins=N)
wdist_fitted.fit(dirs,speeds)

In [ ]:
fig,ax = plt.subplots(nrows=2)
ax[0].hist(speeds,density=True,label='Simulated',bins=None)
x = np.linspace(ax[0].get_xlim()[0],ax[0].get_xlim()[1],1000)
ax[0].plot(x,wdist2.pdf(x),'r--',label='True distribution')
ax[0].plot(x,wdist_fitted.pdf(x),'g:',label='Estimated distribution')
dir_bins = np.digitize(dirs,wdist2.azimuth_bin_edges,right=True)
counts = np.bincount(dir_bins,minlength=len(probabilities))
ax[1].bar(x=wdist2.azimuth_bin_centers,width=360/N,height=counts/sum(counts),label='Simulated')
ax[1].bar(x=wdist2.azimuth_bin_centers,height=probabilities,width=360/2/N,color="red",alpha=0.3,label='True distribution')
ax[1].bar(x=wdist2.azimuth_bin_centers,height=wdist_fitted.probabilities,width=360/4/N,color="green",alpha=0.3,label='Estimated distribution')
ax[0].legend()
ax[1].legend()